In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
!pip install textblob 
 
import nltk ,xgboost,numpy,textblob,string
from nltk.corpus import stopwords

from sklearn import model_selection , preprocessing, linear_model, naive_bayes, metrics
from sklearn import decomposition,ensemble
from sklearn import decomposition,ensemble

from keras.preprocessing import text,sequence
from keras import layers,models,optimizers

In [2]:
data  = pd.read_csv("train.tsv", sep  ="\t")

In [3]:
data.head() 
# Sentiment skorlarımız var aslında ama görmezden gelip tekrar sınıflandıracagız 
# ML problemlerinde en ciddi problemlerden birisi de sınıflandırması belli olan veri setlerine ulaşmaktır ancak burada sınıflandırılmış
# ancak biz bunu OLUMLU ya da OLUMSUZ olarak sınıflandıracağız


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
data["Sentiment"].sample(5)

128901    1
51055     2
150294    2
86577     2
48886     1
Name: Sentiment, dtype: int64

In [5]:
data["Sentiment"].replace(0, value = "negatif", inplace = True)  # 0 ve 1 olanları negatif yaptım 
data["Sentiment"].replace(1, value = "negatif", inplace = True)

In [6]:
data["Sentiment"].replace(3, value = "pozitif", inplace = True)  # 3 ve 4  olanları pozitif yaptım 
data["Sentiment"].replace(4, value = "pozitif", inplace = True)

In [7]:
data.sample(5) # sınıflandırmayı daha keskın yapabilmek adına 2 olanları yani arada kalanları cıkaracağım 

,PhraseId,SentenceId,Phrase,Sentiment
7872,7873,320,movie structures,2
14648,14649,631,in the last few cloying moments,negatif
125387,125388,6737,anxieties,2
140010,140011,7596,wins you,pozitif
52941,52942,2624,economical,pozitif


In [8]:
data  = data[(data.Sentiment== "negatif") | (data.Sentiment == "pozitif")]

In [9]:
data.sample(5)

,PhraseId,SentenceId,Phrase,Sentiment
100392,100393,5267,won my heart,pozitif
96864,96865,5063,", Spirited Away is a triumph of imagination",pozitif
39830,39831,1903,that might have made it an exhilarating,pozitif
62923,62924,3183,belly laugh,pozitif
116322,116323,6201,none of them memorable,negatif


In [10]:
data.Sentiment.value_counts()

pozitif    42133
negatif    34345
Name: Sentiment, dtype: int64

In [11]:
df = pd.DataFrame()

In [12]:
df["text"] = data["Phrase"]
df["label"] = data["Sentiment"]
df.head()

,text,label
0,A series of escapades demonstrating the adage ...,negatif
21,good for the goose,pozitif
22,good,pozitif
33,"the gander , some of which occasionally amuses...",negatif
46,amuses,pozitif


In [13]:
# büyük küçük dönüşümü 
df["text"] = df["text"].apply(lambda x : " ".join(x.lower() for x in x.split())) 

In [14]:
# noktalama işaretleri
df["text"] = df["text"].str.replace('[^\w\s]', '')

In [15]:
# sayılar 
df["text"] = df["text"].str.replace('\d','')

In [16]:
import nltk

In [17]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\APO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
sw = stopwords.words("english")
df["text"] = df["text"].apply(lambda x : " ".join(x for x in x.split() if x not in sw )) 

In [19]:
# seyreklerin silinmesi

sil = pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]
df["text"] = df["text"].apply(lambda x : " ".join(x for x in x.split() if x not in sil )) 

In [20]:
# lemmi

In [21]:
from textblob import Word
df["text"] = df["text"].apply(lambda x : " ".join([Word(word).lemmatize() for word in x.split()]) )

Feature **Engineering**

In [22]:
df.head()

,text,label
0,series demonstrating adage good goose also goo...,negatif
21,good goose,pozitif
22,good,pozitif
33,gander occasionally amuses none amount much story,negatif
46,amuses,pozitif


count vectors

In [23]:
df.iloc[0]

text     series demonstrating adage good goose also goo...
label                                              negatif
Name: 0, dtype: object

In [24]:
# tf = (bir t teriminin bir dökümanda gözlenme frekansı)/(dökümandaki toplam terim sayısı)

In [25]:
# idf = log_e(toplam döküman sayısı / icinde t terimi olan belge sayısı)

In [26]:
# word embeddings : kelime ve dökümanların yoğunluk vectörüdür

**test** - **train**

In [27]:
x_train,x_test,y_train , y_test = model_selection.train_test_split(
                                                                    df["text"],  # bagımsız değişken
                                                                    df["label"]  # bagımlı değişken
                                                                   )

In [28]:
encoder = preprocessing.LabelEncoder()
                                    # string formatta olan bağımlı/bagımsız değişkenmzi dönüştürmüş olduk
y_train = encoder.fit_transform(y_train)
y_test =  encoder.fit_transform(y_test)

In [29]:
y_test

array([0, 1, 0, ..., 0, 0, 1])

In [30]:
y_train

array([0, 0, 1, ..., 1, 0, 1])

count vectors

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
vectorizer = CountVectorizer()
X_train_count = vectorizer.fit_transform(x_train)
X_test_count = vectorizer.transform(x_test)

In [33]:
vectorizer.get_feature_names()[:5]

['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned']

Count vektör yonteminde her bir satır , yorumu
her bir sütunda da her bir dökümandaki kelimenin geçme frekansını temsil eder demiştik 



In [34]:
X_train_count.toarray() # unique olanları alacağı için matrisimde 0 lardan oluşacağını görmüş olduk 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

**tf-idf**

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(x_train)

TfidfVectorizer()

In [37]:
X_train_tf_idf_word = tf_idf_word_vectorizer.fit_transform(x_train)
X_testtf_idf_word = tf_idf_word_vectorizer.transform(x_test)

In [38]:
tf_idf_word_vectorizer.get_feature_names()[:5]

['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned']

**MAKİNE ÖĞRENMESİ İLE SENTİMENT SINIFLANDIRMASI**

In [46]:
from sklearn.model_selection import train_test_split,cross_val_score

In [50]:
####  Count vektörize ile loj. Reg. modeli kullanılarak tahmin alıyorum

In [51]:
loj = linear_model.LogisticRegression()  
loj_model = loj.fit(X_train_count, y_train)
accuracy = model_selection.cross_val_score(loj_model,
                                           X_test_count,
                                           y_test,
                                           cv = 10).mean()

In [52]:
print("Count Vectors Doğruluk Oranı : ", accuracy)

Count Vectors Doğruluk Oranı :  0.837970711297071


In [60]:
rf= ensemble.RandomForestClassifier()
rf_model = rf.fit(X_train_count, y_train)
accuracy = model_selection.cross_val_score(rf_model,
                                           X_test_count,
                                           y_test,
                                           cv = 10).mean()
print("Count Vectors Doğruluk Oranı : ", accuracy)

Count Vectors Doğruluk Oranı :  0.8299163179916318


In [61]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(X_train_count, y_train)
accuracy = model_selection.cross_val_score(xgb_model,
                                           X_test_count,
                                           y_test,
                                           cv = 10).mean()
print("Count Vectors Doğruluk Oranı : ", accuracy)

[19:12:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:12:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:12:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:12:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel